<a href="https://colab.research.google.com/github/steven1174/Web_Scraping_with_Python/blob/main/C3_Starting_to__Crawl_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
%%capture
pip install twitter

In [ ]:
%%capture
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import re

# Twitter

In [ ]:
from twitter import Twitter

In [ ]:
Twitter?

In [ ]:
Twitter(auth=OAuth(<Access Token>, <Access Token Secret>, <Consumer Key>,<Consumer Secret>)))

In [ ]:
%%capture
html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon")
bsObj = BeautifulSoup(html)

for link in bsObj.findAll("a"):
  if 'href' in link.attrs:
    print(link.attrs['href'])

In [ ]:
%%capture
html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon")
bsObj = BeautifulSoup(html)

for link in bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$")):
  if 'href' in link.attrs:
    print(link.attrs['href'])

In [ ]:
import datetime
import random
import re

In [ ]:
random.seed(datetime.datetime.now())

def getLinks(articleUrl):
  html = urlopen("http://en.wikipedia.org" + articleUrl)
  bsObj = BeautifulSoup(html)
  return bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$"))

In [ ]:
links = getLinks("/wiki/Kevin_Bacon")

In [ ]:
# while len(links) > 0:
#  newArticle = newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
#  print(newArticle)
#  links = getLinks(newArticle)

# Crawling an Entire Site

In [ ]:
pages = set()

def getLinks(pageUrl):

  global pages
  html = urlopen("http://en.wikipedia.org" + pageUrl)
  bsObj = BeautifulSoup(html)

  for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
    if 'href' in link.attrs:
      if link.attrs['href'] not in pages:
        newPage = link.attrs['href']
        print(newPage)

        pages.add(newPage)
        getLinks(newPage)

In [ ]:
# getLinks("")

In [ ]:
pages = set()

def getLinks(pageUrl):
  global pages
  html = urlopen("http://en.wikipedia.org" + pageUrl)
  bsObj = BeautifulSoup(html)

  try:
    print(bsObj.h1.get_text())
    print(bsObj.find(id ="mw-content-text").findAll("p")[0])
    print(bsObj.find(id="ca-edit").find("span").find("a").attrs['href'])
  except AttributeError:
    print("This page is missing something! No worries though!")
    
  for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
    if 'href' in link.attrs:
      if link.attrs['href'] not in pages:
        newPage = link.attrs['href']
        print("----------------\n"+newPage)
        pages.add(newPage)
        getLinks(newPage)

In [ ]:
# getLinks("")

# Crawling Across the Internet

In [ ]:
import re
import datetime
import random

In [ ]:
pages = set()
random.seed(datetime.datetime.now())

#Retrieves a list of all Internal links found on a page
def getInternalLinks(bsObj, includeUrl):

  internalLinks = []

  #Finds all links that begin with a "/"
  for link in bsObj.findAll("a", href=re.compile("^(/|.*"+ includeUrl+ ")")):
    if link.attrs['href'] is not None:
      if link.attrs['href'] not in internalLinks:
        internalLinks.append(link.attrs['href'])
  
  return internalLinks

#Retrieves a list of all external links found on a page
def getExternalLinks(bsObj, excludeUrl):
  
  externalLinks = []

  #Finds all links that start with "http" or "www" that do
  #not contain the current URL
  for link in bsObj.findAll("a", href=re.compile("^(http|www)((?!" + excludeUrl+ ").)*$")):
    if link.attrs['href'] is not None:
      if link.attrs['href'] not in externalLinks:
        externalLinks.append(link.attrs['href'])
    
  return externalLinks

def splitAddress(address):
  addressParts = address.replace("http://", "").split("/")
  return addressParts

def getRandomExternalLink(startingPage):
  html = urlopen(startingPage)
  bsObj = BeautifulSoup(html)
  externalLinks = getExternalLinks(bsObj, splitAddress(startingPage)[0])
  
  if len(externalLinks) == 0:
    internalLinks = getInternalLinks(startingPage)
    return getNextExternalLink(internalLinks[random.randint(0,
    len(internalLinks)-1)])
  else:
    return externalLinks[random.randint(0, len(externalLinks)-1)]

def followExternalOnly(startingSite):
  externalLink = getRandomExternalLink("http://oreilly.com")
  print("Random external link is: "+externalLink)
  followExternalOnly(externalLink)

In [ ]:
# followExternalOnly("http://oreilly.com")

In [ ]:
#Collects a list of all external URLs found on the site
allExtLinks = set()
allIntLinks = set()

def getAllExternalLinks(siteUrl):
  html = urlopen(siteUrl)
  bsObj = BeautifulSoup(html)
  internalLinks = getInternalLinks(bsObj,splitAddress(siteUrl)[0])
  externalLinks = getExternalLinks(bsObj,splitAddress(siteUrl)[0])

  for link in externalLinks:
    if link not in allExtLinks:
      allExtLinks.add(link)
      print(link)

  for link in internalLinks:
    if link not in allIntLinks:
      print("About to get link: " + link)
      allIntLinks.add(link)
      getAllExternalLinks(link)

In [ ]:
# getAllExternalLinks("http://oreilly.com")

# Crawling with Scrapy

In [ ]:
%%capture
pip install scrapy

In [ ]:
from scrapy import Item, Field

In [ ]:
class Article(Item):
  # define the fields for your item here like:
  # name = scrapy.Field()
  title = Field()